# Create an Agent with a single Knowledge Base only

In this notebook you will learn how to create an Amazon Bedrock Agent that connects to a single Knowledge Bases for Amazon Bedrock to retrieve company data and complete tasks. 

The use case for this notebook is the Amazon Bedrock Documentation pages stored as PDFs. It will allow you to ask questions about Amazon Bedrock and get answers based on documents available in the Knowledge Base.

The steps to complete this notebook are:

1. Import the needed libraries
1. Create an S3 bucket and upload the data to it
1. Create the Knowledge Base for Amazon Bedrock and sync data to Knowledge Base
1. Create the Agent for Amazon Bedrock
1. Test the Agent
1. Clean up the resources created

## 1. Import the needed libraries

In [1]:
!pip install --upgrade -q opensearch-py
!pip install --upgrade -q requests-aws4auth
!pip install --upgrade -q boto3
!pip install --upgrade -q botocore
#!pip install --upgrade -q awscli

In [2]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

[2024-10-03 11:03:35,461] p1019363 {credentials.py:1147} INFO - Found credentials in environment variables.


In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '864981754328')

In [6]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "bedrock-docs-kb-agents"
agent_alias_name = "bedrock-docs-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_docs'
kb_name = f'bedrock-docs-kb-{suffix}'
data_source_name = f'bedrock-docs-kb-docs-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"
kb_collection_name = f'bd-kbc-{suffix}'
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to Bedrock's documentation in a Knowledge Base
and you can Answer questions from this documentation. Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

## 2. Upload the dataset to Amazon S3
Knowledge Bases for Amazon Bedrock, currently require data to reside in an Amazon S3 bucket. In this section we will create an Amazon S3 bucket and the files.

### 2.1 Create the Amazon S3 bucket

In [7]:
if region != 'us-east-1':
    s3_client.create_bucket(
        Bucket=bucket_name.lower(),
        CreateBucketConfiguration={'LocationConstraint': region}
    )
else:
    s3_client.create_bucket(Bucket=bucket_name)

### 2.2 Upload dataset to the Amazon S3 bucket

In [8]:
# Upload Knowledge Base files to this s3 bucket
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

## 3. Create a Knowledge Base for Amazon Bedrock

In this section we will go through all the steps to create and test a Knowledge Base. 

These are the steps to complete:
    
1. Create a Knowledge Base Role and its policies
1. Create a Vector Database
1. Create an OpenSearch Index
1. Create a Knowledge Base
1. Create a data source and attach to the recently created Knowledge Base
1. Ingest data to your knowledge Base

### 3.1 Create Knowledge Base Role and Policies

Let's first create IAM policies to allow our Knowledge Base to access Bedrock Titan Embedding Foundation model, Amazon OpenSearch Serverless and the S3 bucket with the Knowledge Base Files.

Once the policies are ready, we will create the Knowledge Base role

In [9]:
# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [10]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [11]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [12]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '7e3f601c-0a51-4d31-a358-8e0e44d1a2ce',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 03 Oct 2024 15:07:17 GMT',
   'x-amzn-requestid': '7e3f601c-0a51-4d31-a358-8e0e44d1a2ce',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [13]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::864981754328:role/AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'

### 3.2 Create Vector Database

Firt of all we have to create a vector store. In this section we will use Amazon OpenSerach Serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.


In [14]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [15]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [16]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:iam::864981754328:user/peteryxu'

In [17]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [18]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-east-1:864981754328:collection/ltwjvxkrsohpvjj1818c',
  'createdDate': 1727968405481,
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'id': 'ltwjvxkrsohpvjj1818c',
  'kmsKeyArn': 'auto',
  'lastModifiedDate': 1727968405481,
  'name': 'bd-kbc-us-east-1-864981754328',
  'standbyReplicas': 'DISABLED',
  'status': 'CREATING',
  'type': 'VECTORSEARCH'},
 'ResponseMetadata': {'RequestId': 'f349e119-5a8d-4d9a-96b9-b9ff4920d2d0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f349e119-5a8d-4d9a-96b9-b9ff4920d2d0',
   'date': 'Thu, 03 Oct 2024 15:13:25 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [19]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:864981754328:collection/ltwjvxkrsohpvjj1818c'

In [20]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host


Collection successfully created:
[{'arn': 'arn:aws:aoss:us-east-1:864981754328:collection/ltwjvxkrsohpvjj1818c', 'collectionEndpoint': 'https://ltwjvxkrsohpvjj1818c.us-east-1.aoss.amazonaws.com', 'createdDate': 1727968405481, 'dashboardEndpoint': 'https://ltwjvxkrsohpvjj1818c.us-east-1.aoss.amazonaws.com/_dashboards', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'id': 'ltwjvxkrsohpvjj1818c', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1727968642496, 'name': 'bd-kbc-us-east-1-864981754328', 'standbyReplicas': 'DISABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


'ltwjvxkrsohpvjj1818c.us-east-1.aoss.amazonaws.com'

### 3.3 - Create OpenSearch Index

Let's now create a vector index to index our data


In [21]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2024-10-03 12:21:52,959] p1019363 {credentials.py:1147} INFO - Found credentials in environment variables.
[2024-10-03 12:22:37,988] p1019363 {base.py:258} INFO - PUT https://ltwjvxkrsohpvjj1818c.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:0.800s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


### 3.5 - Create Knowledge Base
Now that we have the Vector database available in OpenSearch Serverless, let's create a Knowledge Base and associate it with the OpenSearch DB

In [22]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [23]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1013',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 03 Oct 2024 16:28:57 GMT',
                                      'x-amz-apigw-id': 'fFPrREEloAMEBbw=',
                                      'x-amzn-requestid': 'e61cedf7-1afd-4372-860b-f73b30a41f23',
                                      'x-amzn-trace-id': 'Root=1-66fec647-3995089d269ed0a75f017d97'},
                      'HTTPStatusCode': 202,
                      'RequestId': 'e61cedf7-1afd-4372-860b-f73b30a41f23',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2024, 10, 3, 16, 28, 55, 901018, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:864981754328:knowledge-base/ZMAW

In [24]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '661',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 03 Oct 2024 16:30:39 GMT',
                                      'x-amz-apigw-id': 'fFP7YGVBoAMEghQ=',
                                      'x-amzn-requestid': 'd8979238-f4ab-4b36-bd80-767f1c898b83',
                                      'x-amzn-trace-id': 'Root=1-66fec6ae-16f20af808f2b9984a3e21a6'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd8979238-f4ab-4b36-bd80-767f1c898b83',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2024, 10, 3, 16, 30, 39, 30217, tzinfo=tzutc()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-docs-kb-agents-us-east-1-86498175432

### 3.6 - Start ingestion job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.


In [25]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)


## 4. Create Agent

We will now create the Agent and associate the Knowledge Base to it. To do so we need to: 
1. Create Agent IAM role and policies
1. Create Agent
1. Associate Agent to Knowledge Base
1. Prepare Agent

### 4.1 - Create Agent IAM role and policies
First we need to create the agent policies that allow bedrock model invocation and Knowledge Base retrieval

In [26]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [27]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)


In [28]:

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '6ed3e42b-b4db-49cc-a410-5a5d23638216',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 03 Oct 2024 16:32:20 GMT',
   'x-amzn-requestid': '6ed3e42b-b4db-49cc-a410-5a5d23638216',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### 4.2 - Create Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [29]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Let's now store the agent id in a local variable to use it on the next steps

In [30]:
agent_id = response['agent']['agentId']
agent_id

'CHGGV5HMUG'

### 4.3 - Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [31]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

### 4.4 - Prepare Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [32]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '0197f9c1-dc9a-4880-bc93-ae0c45608368',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 03 Oct 2024 16:33:24 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0197f9c1-dc9a-4880-bc93-ae0c45608368',
   'x-amz-apigw-id': 'fFQVRHcQIAMEl5w=',
   'x-amzn-trace-id': 'Root=1-66fec754-1d0f200f575e76a37a5b81fe'},
  'RetryAttempts': 0},
 'agentId': 'CHGGV5HMUG',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 10, 3, 16, 33, 24, 731699, tzinfo=tzutc())}

## 5 - Testing Agent

Now that we have our agent, let's invoke it to test if it is providing correct information about Amazon Bedrock. To do so, let's first create an Agent Alias

In [33]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [34]:
agent_alias

{'ResponseMetadata': {'RequestId': '398d5f13-9444-4d78-bd21-c67decc1c598',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 03 Oct 2024 16:34:11 GMT',
   'content-type': 'application/json',
   'content-length': '344',
   'connection': 'keep-alive',
   'x-amzn-requestid': '398d5f13-9444-4d78-bd21-c67decc1c598',
   'x-amz-apigw-id': 'fFQchF6oIAMEsQA=',
   'x-amzn-trace-id': 'Root=1-66fec783-6d3aaeb16ecb42bd6271385a'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:864981754328:agent-alias/CHGGV5HMUG/C075XGKMDI',
  'agentAliasId': 'C075XGKMDI',
  'agentAliasName': 'bedrock-docs-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'CHGGV5HMUG',
  'createdAt': datetime.datetime(2024, 10, 3, 16, 34, 11, 88888, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 10, 3, 16, 34, 11, 88888, tzinfo=tzutc())}}

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and get the information from the Knowledge base

In [35]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How can I evaluate models on Bedrock?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))


[2024-10-03 12:35:19,813] p1019363 {2021949151.py:19} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 03 Oct 2024 16:35:19 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '7a8266ac-81a5-11ef-a932-2f18ffe82f68',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '4db03a12-d5a2-4acf-aea5-e804020994f7'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4db03a12-d5a2-4acf-aea5-e804020994f7',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fcf0c194970>,
 'contentType': 'application/json',
 'sessionId': '7a8266ac-81a5-11ef-a932-2f18ffe82f68'}


In [36]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-10-03 12:35:28,770] p1019363 {<timed exec>:11} INFO - {
  "agentAliasId": "C075XGKMDI",
  "agentId": "CHGGV5HMUG",
  "agentVersion": "1",
  "sessionId": "7a8266ac-81a5-11ef-a932-2f18ffe82f68",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "ZMAWZL4NJA",
          "text": "How can I evaluate models on Bedrock?"
        },
        "traceId": "4db03a12-d5a2-4acf-aea5-e804020994f7-0"
      }
    }
  }
}
[2024-10-03 12:35:28,775] p1019363 {<timed exec>:11} INFO - {
  "agentAliasId": "C075XGKMDI",
  "agentId": "CHGGV5HMUG",
  "agentVersion": "1",
  "sessionId": "7a8266ac-81a5-11ef-a932-2f18ffe82f68",
  "trace": {
    "orchestrationTrace": {
      "observation": {
        "knowledgeBaseLookupOutput": {
          "retrievedReferences": [
            {
              "content": {
                "text": "Working with model evaluation jobs in Amazon Bedrock   T

CPU times: user 18.8 ms, sys: 921 μs, total: 19.7 ms
Wall time: 3.58 s


In [37]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

You can evaluate models on Amazon Bedrock using the Model Evaluation feature. With Model Evaluation, you can evaluate and compare the output of different models to determine which one is best suited for your application. You can create either an automatic model evaluation job or a human worker-based model evaluation job. For an automatic model evaluation job, you need to specify the models to evaluate, the task type (e.g. text generation), metrics and datasets to use, and where to store the results. You also need to create or use an existing IAM service role with the required permissions. You can create an automatic model evaluation job through the Amazon Bedrock console, AWS CLI, or SDKs. For a human worker-based model evaluation job, you create a work team of human workers who will evaluate the model outputs based on your instructions and rating criteria. You provide instructions and preview the evaluation UI when setting up the job. These jobs allow for more subjective evaluation of

In [38]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    if session_id is None:
        session_id:str = str(uuid.uuid1())

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    agent_answer = ''
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    return agent_answer

In [39]:
simple_agent_invoke("what is bedrock provisioned throughput?", agent_id, agent_alias_id, session_id)

[2024-10-03 12:35:58,895] p1019363 {321621934.py:13} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Thu, 03 Oct 2024 16:35:58 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '7a8266ac-81a5-11ef-a932-2f18ffe82f68',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '00aa1a95-043a-4075-aa7a-c2b8f3e775e0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '00aa1a95-043a-4075-aa7a-c2b8f3e775e0',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fcf0c195390>,
 'contentType': 'application/json',
 'sessionId': '7a8266ac-81a5-11ef-a932-2f18ffe82f68'}


[2024-10-03 12:36:17,136] p1019363 {321621934.py:21} INFO - Final answer ->
Amazon Bedrock provisioned throughput allows you to purchase a guaranteed level of throughput for running inference on Amazon Bedrock models. With provisioned throughput, you commit to a certain number of model units (MUs) for a set duration (no commitment, 1 month, or 6 months). The longer the commitment duration, the more discounted the hourly price becomes. Provisioned throughput helps ensure consistent performance and availability for your model inference workloads.


'Amazon Bedrock provisioned throughput allows you to purchase a guaranteed level of throughput for running inference on Amazon Bedrock models. With provisioned throughput, you commit to a certain number of model units (MUs) for a set duration (no commitment, 1 month, or 6 months). The longer the commitment duration, the more discounted the hourly price becomes. Provisioned throughput helps ensure consistent performance and availability for your model inference workloads.'

In [ ]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

In [ ]:
simple_agent_invoke("what are the components of a Bedrock Guardrail?", agent_id, agent_alias_id, session_id)

## 6 - Clean up (Optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. delete agent alias
1. delete agent
1. delete the knowledge base
1. delete the OpenSearch Serverless vector store
1. empty created s3 bucket
1. delete s3 bucket


In [ ]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
# Delete IAM Roles and policies and Knowledge Base files
for policy in [
    agent_bedrock_policy, 
    agent_kb_schema_policy,
    kb_bedrock_policy,
    kb_aoss_policy,
    kb_s3_policy
]:
    response = iam_client.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam_client.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

    

for role_name in [
    agent_role_name, 
    kb_role_name
]:
    try: 
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        
    
try:

    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )

    open_search_serverless_client.delete_access_policy(
          name=kb_collection_name,
          type='data'
    )    

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='network'
    )   

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='encryption'
    )    
    bedrock_agent_client.delete_knowledge_base(
        knowledgeBaseId=knowledge_base_id
    )
except Exception as e:
    print(e)

## Conclusion

We have now experimented with using boto3 SDK to create, invoke and delete an agent having a single KB connected to it.
## Take aways

Adapt this notebook to create new agents for your application

## Thank You